In [1]:
!pip install kats


     |████████████████████████████████| 6.3 MB 17.4 MB/s 
     |████████████████████████████████| 815 kB 38.4 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 498 kB 41.1 MB/s 
     |████████████████████████████████| 3.4 MB 36.3 MB/s 
     |████████████████████████████████| 20.6 MB 1.4 MB/s 
     |████████████████████████████████| 9.5 MB 43.3 MB/s 
     |████████████████████████████████| 25.3 MB 61 kB/s 
     |████████████████████████████████| 475 kB 53.0 MB/s 
  Created wheel for kats: filename=kats-0.1.0-py3-none-any.whl size=286608 sha256=351252cc188f8fcab6cb0dfe1edb5da9568b65a4ce4573ef1ef4cea65977a294
  Stored in directory: /root/.cache/pip/wheels/c8/dd/5b/cc7cb7fc37c5b838c65e504437bba4c1828a3fbb473c0c11be
  Created wheel for fbprophet: filename=fbprophet-0.7-py3-none-any.whl size=6637707 sha256=822d1416fa203689c16ec38df7f9da38a145bda19068140c07035d64f48d2dd7
  Stored in directory: /root/.cache/pip/wheels/82/e8/a8/53f37f0a409bc51

## Examine the time series to find the most volatile sample, and find zones that are similar to each other via clustering
* using Colab because Kats couldn't be installed on paperspace instance

In [13]:
import numpy as np
import pandas as pd
import os
from  datetime import datetime
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from kats.consts import TimeSeriesData
from statsmodels.tsa.seasonal import STL
from kats.utils.simulator import Simulator
from sklearn.preprocessing import StandardScaler
from kats.tsfeatures.tsfeatures import TsFeatures

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir('/content/drive/My Drive/galvanize_datathon')

###  extract time series features for entropy, which corresponds to volatility
*  For each month, select time series from each zone
*  For each zone, select time series (didn't get to this part due to insufficient RAM on COLAB).

---





In [7]:
# get the data
price_path='Jan2021-July2021-price.csv'

price_df=pd.read_csv(price_path, parse_dates=['Time Stamp'])
price_df.index = pd.to_datetime(price_df['Time Stamp'])

In [8]:
price_df.head()

,Time Stamp,Name,PTID,LBMP ($/MWHr),Marginal Cost Losses ($/MWHr),Marginal Cost Congestion ($/MWHr),weekday,hour,month,dayofweek
Time Stamp,,,,,,,,,,
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,CAPITL,61757,28.83,2.06,0.0,4,0,1,Friday
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,CENTRL,61754,27.11,0.35,0.0,4,0,1,Friday
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,DUNWOD,61760,29.02,2.25,0.0,4,0,1,Friday
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,GENESE,61753,26.29,-0.48,0.0,4,0,1,Friday
2021-01-01 00:05:00+00:00,2021-01-01 00:05:00+00:00,H Q,61844,25.99,-0.78,0.0,4,0,1,Friday


In [9]:
def monthly_ts(df):
  zonal_df=[]
  for zone in df['Name'].unique():
    month_ts_ls=[]
    for i in df['month'].unique():
      sub_df=df.loc[(df['Name']==zone) & (df['month']==i)]
      ts=sub_df[['LBMP ($/MWHr)']].reset_index()
      ts.columns=['time', 'value']
      month_ts_ls.append(ts)

    model = TsFeatures()
    output_features = [model.transform(ts) for ts in month_ts_ls] 
    df_features = pd.DataFrame(output_features)
    df_features['month_int']=[1, 2, 3, 4, 5, 6, 7]
    df_features['zone']=zone
    zonal_df.append(df_features)
  
  zonal_df=pd.concat(zonal_df, axis=0).reset_index()
  return zonal_df

In [6]:
def zonal_ts(df):
  zonal_df=[]

  for zone in df['Name'].unique():
    
      sub_df=df.loc[df['Name']==zone]
      ts=sub_df[['LBMP ($/MWHr)']].reset_index()
      ts.columns=['time', 'value']

      model = TsFeatures()
      output_features = model.transform(ts)
      df_features = pd.DataFrame(output_features)
      df_features['zone']=zone
      zonal_df.append(df_features)
  
  zonal_df=pd.concat(zonal_df, axis=0)
  return zonal_df


In [ ]:
# cannot run this because of RAM constraint on COLAB!
zonal_df=zonal_ts(price_df)

In [10]:
month_df=monthly_ts(price_df)

/usr/local/lib/python3.7/dist-packages/kats/tsfeatures/tsfeatures.py:396: FutureWarning:

the 'unbiased'' keyword is deprecated, use 'adjusted' instead

/usr/local/lib/python3.7/dist-packages/kats/tsfeatures/tsfeatures.py:403: FutureWarning:

the 'unbiased'' keyword is deprecated, use 'adjusted' instead

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning:

After 0.13 initialization must be handled at model creation

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/stattools.py:1907: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


/usr/local/lib/python3.7/dist-packages/kats/tsfeatures/tsfeatures.py:396: FutureWarning:

the 'unbiased'' keyword is deprecated, use 'adjusted' instead

/usr/local/lib/python3.7/dist-packages/kats/tsfeatures/tsfeatures.py:403: FutureWarning:

the 'unbiased'' keyword is deprecated, use 'adjust

In [15]:
month_df['month']=month_df['month_int'].apply(lambda x: datetime(year=2021, month=x, day=2))

In [16]:
month_df.tail()

,index,length,mean,var,entropy,lumpiness,stability,flat_spots,hurst,std1st_der,crossing_points,binarize_mean,unitroot_kpss,heterogeneity,histogram_mode,linearity,trend_strength,seasonality_strength,spikiness,peak,trough,level_shift_idx,level_shift_size,y_acf1,y_acf5,diff1y_acf1,diff1y_acf5,diff2y_acf1,diff2y_acf5,y_pacf5,diff1y_pacf5,diff2y_pacf5,seas_acf1,seas_pacf1,firstmin_ac,firstzero_ac,holt_alpha,holt_beta,hw_alpha,hw_beta,hw_gamma,month_int,zone,month
100,2,9100,15.601151,834.192209,0.811071,9.782897e+06,350.792381,27,0.109086,12.663098,581,0.379670,1.932796,2517.041408,-24.920,0.016231,0.769847,0.335142,0.059384,1,3,1976,33.9180,0.735820,1.453888,-0.272308,0.095369,-0.594279,0.373084,0.576960,0.127835,0.666927,0.311729,0.072219,8,488,0.664989,0.000100,NaN,NaN,NaN,3,WEST,2021-03-02
101,3,8827,10.796111,988.507173,0.934719,1.691675e+08,195.077769,17,0.044909,20.723628,903,0.464824,2.236251,2926.233970,-163.965,0.000200,0.515029,0.319801,1.553739,0,4,8433,74.0640,0.400131,0.380508,-0.275725,0.340871,-0.455020,0.411344,0.296760,0.366963,0.951422,0.054051,-0.100691,2,417,0.240812,0.011558,NaN,NaN,NaN,4,WEST,2021-04-02
102,4,9157,18.636351,314.584795,0.825940,3.734622e+06,129.028942,22,0.061723,9.024749,739,0.536639,0.895857,3079.489639,-38.311,0.013101,0.698204,0.350651,0.036973,1,6,3643,32.7560,0.664103,1.086687,-0.229226,0.082272,-0.537312,0.293107,0.471568,0.165077,0.678734,0.324674,0.065912,38,159,0.510220,0.000095,NaN,NaN,NaN,5,WEST,2021-05-02
103,5,8813,32.051319,1711.427280,0.717324,5.137679e+07,1081.512419,8,0.085590,16.267041,530,0.299898,8.061462,518.444572,-93.928,0.050498,0.815649,0.333809,2.133706,2,4,3554,87.1435,0.735357,2.213076,-0.415731,0.173851,-0.640357,0.429344,0.701901,0.274378,0.882086,0.580605,0.062458,8,417,0.311425,0.000100,NaN,NaN,NaN,6,WEST,2021-06-02
104,6,7306,34.568368,1204.523810,0.741064,4.705918e+06,725.598079,8,0.102133,14.216837,528,0.282371,2.324818,1458.542853,2.172,0.000010,0.803040,0.325774,0.220069,0,2,1995,43.4595,0.746926,2.044041,-0.336944,0.121130,-0.601485,0.372899,0.656952,0.208726,0.789261,0.534519,0.053319,14,114,0.406887,0.015843,NaN,NaN,NaN,7,WEST,2021-07-02


In [12]:
month_df.to_csv('price_volatility_jan_july.csv', index=False)